In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf

In [11]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [2]:
FILE_PATH = "/kaggle/input/stroke-prediction-dataset/healthcare-dataset-stroke-data.csv"
FEATURES = ["age", "hypertension", "heart_disease", "ever_married", "work_type", "Residence_type", "avg_glucose_level", "bmi", "smoking_status", "stroke"]
CLASS_NAME = ["safe", "risky"]

In [3]:
dataset = pd.read_csv(FILE_PATH)
dataset

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1
...,...,...,...,...,...,...,...,...,...,...,...,...
5105,18234,Female,80.0,1,0,Yes,Private,Urban,83.75,NaN,never smoked,0
5106,44873,Female,81.0,0,0,Yes,Self-employed,Urban,125.20,40.0,never smoked,0
5107,19723,Female,35.0,0,0,Yes,Self-employed,Rural,82.99,30.6,never smoked,0
5108,37544,Male,51.0,0,0,Yes,Private,Rural,166.29,25.6,formerly smoked,0


In [4]:
dataset.drop(columns=["id"], inplace=True)
dataset.dropna(inplace=True)

In [5]:
dataset.loc[dataset["gender"] == "Other", "gender"] = 2
dataset.loc[dataset["gender"] == "Male", "gender"] = 1
dataset.loc[dataset["gender"] == "Female", "gender"] = 0

dataset.loc[dataset["ever_married"] == "Yes", "ever_married"] = 1
dataset.loc[dataset["ever_married"] == "No", "ever_married"] = 0

dataset.loc[dataset["work_type"] == "Never_worked", "work_type"] = 4
dataset.loc[dataset["work_type"] == "children", "work_type"] = 3
dataset.loc[dataset["work_type"] == "Private", "work_type"] = 2
dataset.loc[dataset["work_type"] == "Self-employed", "work_type"] = 1
dataset.loc[dataset["work_type"] == "Govt_job", "work_type"] = 0

dataset.loc[dataset["Residence_type"] == "Urban", "Residence_type"] = 1
dataset.loc[dataset["Residence_type"] == "Rural", "Residence_type"] = 0

dataset.loc[dataset["smoking_status"] == "never smoked", "smoking_status"] = 3
dataset.loc[dataset["smoking_status"] == "formerly smoked", "smoking_status"] = 2
dataset.loc[dataset["smoking_status"] == "smokes", "smoking_status"] = 1
dataset.loc[dataset["smoking_status"] == "Unknown", "smoking_status"] = 0


print(dataset.gender.unique())
print(dataset.ever_married.unique())
print(dataset.work_type.unique())
print(dataset.Residence_type.unique())
print(dataset.smoking_status.unique())

[1 0 2]
[1 0]
[2 1 0 3 4]
[1 0]
[2 3 1 0]


In [6]:
dataset.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
age,4909.0,42.865374,22.555115,0.08,25.00,44.00,60.00,82.00
hypertension,4909.0,0.091872,0.288875,0.00,0.00,0.00,0.00,1.00
heart_disease,4909.0,0.049501,0.216934,0.00,0.00,0.00,0.00,1.00
avg_glucose_level,4909.0,105.305150,44.424341,55.12,77.07,91.68,113.57,271.74
bmi,4909.0,28.893237,7.854067,10.30,23.50,28.10,33.10,97.60
stroke,4909.0,0.042575,0.201917,0.00,0.00,0.00,0.00,1.00


In [9]:
scaler = MinMaxScaler()
for col in dataset[FEATURES].columns:
    if col != "stroke":
        dataset[col] = scaler.fit_transform(np.array(dataset[col]).reshape(-1, 1))

In [10]:
dataset.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
age,4909.0,0.522282,0.275331,0.0,0.304199,0.536133,0.731445,1.0
hypertension,4909.0,0.091872,0.288875,0.0,0.000000,0.000000,0.000000,1.0
heart_disease,4909.0,0.049501,0.216934,0.0,0.000000,0.000000,0.000000,1.0
ever_married,4909.0,0.652679,0.476167,0.0,0.000000,1.000000,1.000000,1.0
work_type,4909.0,0.432777,0.217037,0.0,0.250000,0.500000,0.500000,1.0
Residence_type,4909.0,0.507232,0.499999,0.0,0.000000,1.000000,1.000000,1.0
avg_glucose_level,4909.0,0.231674,0.205080,0.0,0.101330,0.168775,0.269827,1.0
bmi,4909.0,0.212981,0.089966,0.0,0.151203,0.203895,0.261168,1.0
smoking_status,4909.0,0.540979,0.420838,0.0,0.000000,0.666667,1.000000,1.0
stroke,4909.0,0.042575,0.201917,0.0,0.000000,0.000000,0.000000,1.0


In [ ]:
def create_model();
    model = Sequential()
    model.add(Dense(input_shape=(data)))